In [1]:
import tensorflow as tf
from rich.console import Console
from keras_flops import get_flops

In [2]:
class Scale_Wilson(tf.keras.layers.Layer):

    def __init__(self):
        super().__init__(name="scale_wilson")
        # Create a non-trainable weight.
        self.scaling = tf.keras.layers.Rescaling(scale=1./127.5, offset=-1)

    def call(self, inputs):

        return self.scaling(inputs)

# FIXME this might needs removal
class Resizing(tf.keras.layers.Layer):

    def __init__(self):
        super().__init__(name="resize_wilson")
        # Create a non-trainable weight.
        self.resizing = tf.keras.layers.Resizing(
            height=512,
            width=512,
            interpolation='bicubic',
            crop_to_aspect_ratio=False,
        )

    def call(self, inputs):
        return self.resizing(inputs)


class AnchorBox:
    """Generates anchor boxes.

    This class has operations to generate anchor boxes for feature maps at
    strides `[8, 16, 32, 64, 128]`. Where each anchor each box is of the
    format `[x, y, width, height]`.

    Attributes:
      aspect_ratios: A list of float values representing the aspect ratios of
        the anchor boxes at each location on the feature map
      scales: A list of float values representing the scale of the anchor boxes
        at each location on the feature map.
      num_anchors: The number of anchor boxes at each location on feature map
      areas: A list of float values representing the areas of the anchor
        boxes for each feature map in the feature pyramid.
      strides: A list of float value representing the strides for each feature
        map in the feature pyramid.
    """

    def __init__(self):
        self.aspect_ratios = [0.5, 1.0, 2.0]
        self.scales = [2 ** x for x in [0, 1 / 3, 2 / 3]]

        self._num_anchors = len(self.aspect_ratios) * len(self.scales)
        self._strides = [2 ** i for i in range(3, 8)]
        self._areas = [x ** 2 for x in [32.0, 64.0, 128.0, 256.0, 512.0]]
        self._anchor_dims = self._compute_dims()

    def _compute_dims(self):
        """Computes anchor box dimensions for all ratios and scales at all levels
        of the feature pyramid.
        """
        anchor_dims_all = []
        for area in self._areas:
            anchor_dims = []
            for ratio in self.aspect_ratios:
                anchor_height = tf.math.sqrt(area / ratio)
                anchor_width = area / anchor_height
                dims = tf.reshape(
                    tf.stack([anchor_width, anchor_height], axis=-1), [1, 1, 2]
                )
                for scale in self.scales:
                    anchor_dims.append(scale * dims)
            anchor_dims_all.append(tf.stack(anchor_dims, axis=-2))
        return anchor_dims_all

    def _get_anchors(self, feature_height, feature_width, level):
        """Generates anchor boxes for a given feature map size and level

        Arguments:
          feature_height: An integer representing the height of the feature map.
          feature_width: An integer representing the width of the feature map.
          level: An integer representing the level of the feature map in the
            feature pyramid.

        Returns:
          anchor boxes with the shape
          `(feature_height * feature_width * num_anchors, 4)`
        """
        rx = tf.range(feature_width, dtype=tf.float32) + 0.5
        ry = tf.range(feature_height, dtype=tf.float32) + 0.5
        centers = tf.stack(tf.meshgrid(rx, ry), axis=-1) * \
            self._strides[level - 3]
        centers = tf.expand_dims(centers, axis=-2)
        centers = tf.tile(centers, [1, 1, self._num_anchors, 1])
        dims = tf.tile(
            self._anchor_dims[level - 3], [feature_height, feature_width, 1, 1]
        )
        anchors = tf.concat([centers, dims], axis=-1)
        return tf.reshape(
            anchors, [feature_height * feature_width * self._num_anchors, 4]
        )

    def get_anchors(self, image_height, image_width):
        """Generates anchor boxes for all the feature maps of the feature pyramid.

        Arguments:
          image_height: Height of the input image.
          image_width: Width of the input image.

        Returns:
          anchor boxes for all the feature maps, stacked as a single tensor
            with shape `(total_anchors, 4)`
        """
        anchors = [
            self._get_anchors(
                tf.math.ceil(image_height / 2 ** i),
                tf.math.ceil(image_width / 2 ** i),
                i,
            )
            for i in range(3, 8)
        ]
        return tf.concat(anchors, axis=0)


def convert_to_corners(boxes):
    """Changes the box format to corner coordinates

    Arguments:
      boxes: A tensor of rank 2 or higher with a shape of `(..., num_boxes, 4)`
        representing bounding boxes where each box is of the format
        `[x, y, width, height]`.

    Returns:
      converted boxes with shape same as that of boxes.
    """
    return tf.concat(
        [boxes[..., :2] - boxes[..., 2:] / 2.0,
            boxes[..., :2] + boxes[..., 2:] / 2.0],
        axis=-1,
    )


class DecodePredictions(tf.keras.layers.Layer):
    """A Keras layer that decodes predictions of the RetinaNet model.

    Attributes:
      num_classes: Number of classes in the dataset
      confidence_threshold: Minimum class probability, below which detections
        are pruned.
      nms_iou_threshold: IOU threshold for the NMS operation
      max_detections_per_class: Maximum number of detections to retain per
       class.
      max_detections: Maximum number of detections to retain across all
        classes.
      box_variance: The scaling factors used to scale the bounding box
        predictions.
    """

    def __init__(
        self,
        num_classes=80,
        confidence_threshold=0.05,
        nms_iou_threshold=0.5,
        max_detections_per_class=100,
        max_detections=100,
        box_variance=[0.1, 0.1, 0.2, 0.2],
        **kwargs
    ):
        super(DecodePredictions, self).__init__(**kwargs)
        self.num_classes = num_classes
        self.confidence_threshold = confidence_threshold
        self.nms_iou_threshold = nms_iou_threshold
        self.max_detections_per_class = max_detections_per_class
        self.max_detections = max_detections

        self._anchor_box = AnchorBox()
        self._box_variance = tf.convert_to_tensor(
            [0.1, 0.1, 0.2, 0.2], dtype=tf.float32
        )

    def _decode_box_predictions(self, anchor_boxes, box_predictions):
        boxes = box_predictions * self._box_variance
        boxes = tf.concat(
            [
                boxes[:, :, :2] * anchor_boxes[:, :, 2:] +
                anchor_boxes[:, :, :2],
                tf.math.exp(boxes[:, :, 2:]) * anchor_boxes[:, :, 2:],
            ],
            axis=-1,
        )
        boxes_transformed = convert_to_corners(boxes)
        return boxes_transformed

    def call(self, images, predictions):
        image_shape = tf.cast(tf.shape(images), dtype=tf.float32)
        anchor_boxes = self._anchor_box.get_anchors(
            image_shape[1], image_shape[2])
        box_predictions = predictions[:, :, :4]
        cls_predictions = tf.nn.sigmoid(predictions[:, :, 4:])
        boxes = self._decode_box_predictions(
            anchor_boxes[None, ...], box_predictions)

        return tf.image.combined_non_max_suppression(
            tf.expand_dims(boxes, axis=2),
            cls_predictions,
            self.max_detections_per_class,
            self.max_detections,
            self.nms_iou_threshold,
            self.confidence_threshold,
            clip_boxes=False,
        )

In [3]:
import tensorflow as tf
from rich.console import Console
from keras_flops import get_flops

In [4]:
detection_model = tf.keras.models.load_model(
    filepath="/home/fsuser/AI_ENGINE/yolov5_tf_original/Yolov5/Ibrahim_Bhaai_model/fdet-d0", custom_objects=None, compile=True, options=None
)

# full model
Console().print(detection_model.summary())

2022-10-25 17:38:16.982163: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-25 17:38:16.982453: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-25 17:38:16.988772: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-25 17:38:16.989047: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-25 17:38:16.989257: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from S

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 512, 512, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 512, 512, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 stem_conv_pad (ZeroPadding2D)  (None, 513, 513, 3)  0           ['normalization[0][0]']    

None

In [5]:
class Scale_Wilson(tf.keras.layers.Layer):

    def __init__(self):
        super().__init__(name="scale_wilson")
        # Create a non-trainable weight.
        self.scaling = tf.keras.layers.Rescaling(scale=1./127.5, offset=-1)

    def call(self, inputs):

        return self.scaling(inputs)

# FIXME this might needs removal
class Resizing_Wilson(tf.keras.layers.Layer):

    def __init__(self):
        super().__init__(name="resize_wilson")
        # Create a non-trainable weight.
        self.resizing = tf.keras.layers.Resizing(
            height=512,
            width=512,
            interpolation='bicubic',
            crop_to_aspect_ratio=False,
        )

    def call(self, inputs):
        return self.resizing(inputs)


class AnchorBox:
    """Generates anchor boxes.

    This class has operations to generate anchor boxes for feature maps at
    strides `[8, 16, 32, 64, 128]`. Where each anchor each box is of the
    format `[x, y, width, height]`.

    Attributes:
      aspect_ratios: A list of float values representing the aspect ratios of
        the anchor boxes at each location on the feature map
      scales: A list of float values representing the scale of the anchor boxes
        at each location on the feature map.
      num_anchors: The number of anchor boxes at each location on feature map
      areas: A list of float values representing the areas of the anchor
        boxes for each feature map in the feature pyramid.
      strides: A list of float value representing the strides for each feature
        map in the feature pyramid.
    """

    def __init__(self):
        self.aspect_ratios = [0.5, 1.0, 2.0]
        self.scales = [2 ** x for x in [0, 1 / 3, 2 / 3]]

        self._num_anchors = len(self.aspect_ratios) * len(self.scales)
        self._strides = [2 ** i for i in range(3, 8)]
        self._areas = [x ** 2 for x in [32.0, 64.0, 128.0, 256.0, 512.0]]
        self._anchor_dims = self._compute_dims()

    def _compute_dims(self):
        """Computes anchor box dimensions for all ratios and scales at all levels
        of the feature pyramid.
        """
        anchor_dims_all = []
        for area in self._areas:
            anchor_dims = []
            for ratio in self.aspect_ratios:
                anchor_height = tf.math.sqrt(area / ratio)
                anchor_width = area / anchor_height
                dims = tf.reshape(
                    tf.stack([anchor_width, anchor_height], axis=-1), [1, 1, 2]
                )
                for scale in self.scales:
                    anchor_dims.append(scale * dims)
            anchor_dims_all.append(tf.stack(anchor_dims, axis=-2))
        return anchor_dims_all

    def _get_anchors(self, feature_height, feature_width, level):
        """Generates anchor boxes for a given feature map size and level

        Arguments:
          feature_height: An integer representing the height of the feature map.
          feature_width: An integer representing the width of the feature map.
          level: An integer representing the level of the feature map in the
            feature pyramid.

        Returns:
          anchor boxes with the shape
          `(feature_height * feature_width * num_anchors, 4)`
        """
        rx = tf.range(feature_width, dtype=tf.float32) + 0.5
        ry = tf.range(feature_height, dtype=tf.float32) + 0.5
        centers = tf.stack(tf.meshgrid(rx, ry), axis=-1) * \
            self._strides[level - 3]
        centers = tf.expand_dims(centers, axis=-2)
        centers = tf.tile(centers, [1, 1, self._num_anchors, 1])
        dims = tf.tile(
            self._anchor_dims[level - 3], [feature_height, feature_width, 1, 1]
        )
        anchors = tf.concat([centers, dims], axis=-1)
        return tf.reshape(
            anchors, [feature_height * feature_width * self._num_anchors, 4]
        )

    def get_anchors(self, image_height, image_width):
        """Generates anchor boxes for all the feature maps of the feature pyramid.

        Arguments:
          image_height: Height of the input image.
          image_width: Width of the input image.

        Returns:
          anchor boxes for all the feature maps, stacked as a single tensor
            with shape `(total_anchors, 4)`
        """
        anchors = [
            self._get_anchors(
                tf.math.ceil(image_height / 2 ** i),
                tf.math.ceil(image_width / 2 ** i),
                i,
            )
            for i in range(3, 8)
        ]
        return tf.concat(anchors, axis=0)


def convert_to_corners(boxes):
    """Changes the box format to corner coordinates

    Arguments:
      boxes: A tensor of rank 2 or higher with a shape of `(..., num_boxes, 4)`
        representing bounding boxes where each box is of the format
        `[x, y, width, height]`.

    Returns:
      converted boxes with shape same as that of boxes.
    """
    return tf.concat(
        [boxes[..., :2] - boxes[..., 2:] / 2.0,
            boxes[..., :2] + boxes[..., 2:] / 2.0],
        axis=-1,
    )


class DecodePredictions(tf.keras.layers.Layer):
    """A Keras layer that decodes predictions of the RetinaNet model.

    Attributes:
      num_classes: Number of classes in the dataset
      confidence_threshold: Minimum class probability, below which detections
        are pruned.
      nms_iou_threshold: IOU threshold for the NMS operation
      max_detections_per_class: Maximum number of detections to retain per
       class.
      max_detections: Maximum number of detections to retain across all
        classes.
      box_variance: The scaling factors used to scale the bounding box
        predictions.
    """

    def __init__(
        self,
        num_classes=80,
        confidence_threshold=0.05,
        nms_iou_threshold=0.5,
        max_detections_per_class=100,
        max_detections=100,
        box_variance=[0.1, 0.1, 0.2, 0.2],
        **kwargs
    ):
        super(DecodePredictions, self).__init__(**kwargs)
        self.num_classes = num_classes
        self.confidence_threshold = confidence_threshold
        self.nms_iou_threshold = nms_iou_threshold
        self.max_detections_per_class = max_detections_per_class
        self.max_detections = max_detections

        self._anchor_box = AnchorBox()
        self._box_variance = tf.convert_to_tensor(
            [0.1, 0.1, 0.2, 0.2], dtype=tf.float32
        )

    def _decode_box_predictions(self, anchor_boxes, box_predictions):
        boxes = box_predictions * self._box_variance
        boxes = tf.concat(
            [
                boxes[:, :, :2] * anchor_boxes[:, :, 2:] +
                anchor_boxes[:, :, :2],
                tf.math.exp(boxes[:, :, 2:]) * anchor_boxes[:, :, 2:],
            ],
            axis=-1,
        )
        boxes_transformed = convert_to_corners(boxes)
        return boxes_transformed

    def call(self, images, predictions):
        image_shape = tf.cast(tf.shape(images), dtype=tf.float32)
        anchor_boxes = self._anchor_box.get_anchors(
            image_shape[1], image_shape[2])
        box_predictions = predictions[:, :, :4]
        cls_predictions = tf.nn.sigmoid(predictions[:, :, 4:])
        boxes = self._decode_box_predictions(
            anchor_boxes[None, ...], box_predictions)

        return tf.image.combined_non_max_suppression(
            tf.expand_dims(boxes, axis=2),
            cls_predictions,
            self.max_detections_per_class,
            self.max_detections,
            self.nms_iou_threshold,
            self.confidence_threshold,
            clip_boxes=False,
        )

In [6]:
inp = tf.keras.Input(
    shape=(None, None, 3),
    name="input",
    dtype=tf.float32,
)
x = Resizing_Wilson()(inp)
x = Scale_Wilson()(x)
predictions = detection_model(x)
# modified_model = tf.keras.Model(inputs=inp, outputs=out)
detections = DecodePredictions(confidence_threshold=0.5)(inp, predictions)
modified_model = tf.keras.Model(inputs=inp, outputs=detections)

In [7]:
modified_model.summary()
# Calculae FLOPS
flops = get_flops(modified_model, batch_size=1)
print(f"FLOPS: {flops / 10 ** 9:.03} G")

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 resize_wilson (Resizing_Wilson  (None, 512, 512, 3)  0          ['input[0][0]']                  
 )                                                                                                
                                                                                                  
 scale_wilson (Scale_Wilson)    (None, 512, 512, 3)  0           ['resize_wilson[0][0]']          
                                                                                              

2022-10-25 17:38:50.505019: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-25 17:38:50.505340: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-25 17:38:50.505514: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 2
2022-10-25 17:38:50.505674: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2022-10-25 17:38:50.506138: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-25 17:38:50.506352: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] su

Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`


Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`



=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================

Doc:
scope: The nodes in the model graph are organized by their names, which is hierarchical like filesystem.
flops: Number of float operations. Note: Please read the implementation for the math behind it.

Profi

Incomplete shape.
Incomplete shape.


In [8]:
modified_model.save(
    filepath="/home/fsuser/AI_ENGINE/yolov5_tf_original/Yolov5/Ibrahim_Bhaai_model/modified_model_nms",
    overwrite=True,
    include_optimizer=True,
    save_format='tf',
    save_traces=True
)

2022-10-25 17:39:41.151328: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /home/fsuser/AI_ENGINE/yolov5_tf_original/Yolov5/Ibrahim_Bhaai_model/modified_model_nms/assets


INFO:tensorflow:Assets written to: /home/fsuser/AI_ENGINE/yolov5_tf_original/Yolov5/Ibrahim_Bhaai_model/modified_model_nms/assets


In [9]:
import wandb
run = wandb.init()
artifact = run.use_artifact('bb_ai-team/Test-Environment/model-balmy-silence-555:v7', type='model')
artifact_dir = artifact.download()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: aidev (bb_ai-team). Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact model-balmy-silence-555:v7, 55.70MB. 4 files... 
wandb:   4 of 4 files downloaded.  
Done. 0:0:0.1


In [10]:
del modified_model

In [11]:
del detection_model

In [12]:
artifact_dir

'./artifacts/model-balmy-silence-555:v7'

In [13]:
model = tf.keras.models.load_model(
    filepath=artifact_dir, custom_objects=None, compile=False, options=None
)

In [14]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 512, 512, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 512, 512, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 stem_conv_pad (ZeroPadding2D)  (None, 513, 513, 3)  0           ['normalization[0][0]']    

In [15]:
detection_model = tf.keras.models.load_model(
    filepath="/home/fsuser/AI_ENGINE/yolov5_tf_original/Yolov5/Ibrahim_Bhaai_model/fdet-d0", custom_objects=None, compile=True, options=None
)

# full model
Console().print(detection_model.summary())

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 512, 512, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 512, 512, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 stem_conv_pad (ZeroPadding2D)  (None, 513, 513, 3)  0           ['normalization[0][0]']    

None

In [16]:
tf.keras.utils.plot_model(
    model,
    to_file='model.png',
    show_shapes=True,
    show_dtype=False,
    show_layer_names=True,
    rankdir='TB',
    expand_nested=False,
    dpi=300,
    layer_range=None,
    show_layer_activations=False
)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [17]:
tf.keras.utils.plot_model(
    model,
    to_file='model.png',
    show_shapes=True,
    show_dtype=False,
    show_layer_names=True,
    rankdir='TB',
    expand_nested=False,
    dpi=300,
    layer_range=None,
    show_layer_activations=False
)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [18]:
!pip install pydot

In [19]:
tf.keras.utils.plot_model(
    model,
    to_file='model.png',
    show_shapes=True,
    show_dtype=False,
    show_layer_names=True,
    rankdir='TB',
    expand_nested=False,
    dpi=300,
    layer_range=None,
    show_layer_activations=False
)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [20]:
!sudo apt install python-pydot python-pydot-ng graphviz

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package python-pydot
E: Unable to locate package python-pydot-ng


In [21]:
tf.keras.utils.plot_model(
    model,
    to_file='model.png',
    show_shapes=True,
    show_dtype=False,
    show_layer_names=True,
    rankdir='TB',
    expand_nested=False,
    dpi=300,
    layer_range=None,
    show_layer_activations=False
)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [22]:
tf.keras.utils.plot_model(
    model,
    to_file='model.png',
    show_shapes=True,
    show_dtype=False,
    show_layer_names=True,
    rankdir='TB',
    expand_nested=False,
    dpi=300,
    layer_range=None,
    show_layer_activations=False
)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [23]:
class Scale_Wilson(tf.keras.layers.Layer):

    def __init__(self):
        super().__init__(name="scale_wilson")
        # Create a non-trainable weight.
        self.scaling = tf.keras.layers.Rescaling(scale=1./127.5, offset=-1)

    def call(self, inputs):

        return self.scaling(inputs)

# FIXME this might needs removal
class Resizing_Wilson(tf.keras.layers.Layer):

    def __init__(self):
        super().__init__(name="resize_wilson")
        # Create a non-trainable weight.
        self.resizing = tf.keras.layers.Resizing(
            height=512,
            width=512,
            interpolation='bicubic',
            crop_to_aspect_ratio=False,
        )

    def call(self, inputs):
        return self.resizing(inputs)


class AnchorBox:
    """Generates anchor boxes.

    This class has operations to generate anchor boxes for feature maps at
    strides `[8, 16, 32, 64, 128]`. Where each anchor each box is of the
    format `[x, y, width, height]`.

    Attributes:
      aspect_ratios: A list of float values representing the aspect ratios of
        the anchor boxes at each location on the feature map
      scales: A list of float values representing the scale of the anchor boxes
        at each location on the feature map.
      num_anchors: The number of anchor boxes at each location on feature map
      areas: A list of float values representing the areas of the anchor
        boxes for each feature map in the feature pyramid.
      strides: A list of float value representing the strides for each feature
        map in the feature pyramid.
    """

    def __init__(self):
        self.aspect_ratios = [0.5, 1.0, 2.0]
        self.scales = [2 ** x for x in [0, 1 / 3, 2 / 3]]

        self._num_anchors = len(self.aspect_ratios) * len(self.scales)
        self._strides = [2 ** i for i in range(3, 8)]
        self._areas = [x ** 2 for x in [32.0, 64.0, 128.0, 256.0, 512.0]]
        self._anchor_dims = self._compute_dims()

    def _compute_dims(self):
        """Computes anchor box dimensions for all ratios and scales at all levels
        of the feature pyramid.
        """
        anchor_dims_all = []
        for area in self._areas:
            anchor_dims = []
            for ratio in self.aspect_ratios:
                anchor_height = tf.math.sqrt(area / ratio)
                anchor_width = area / anchor_height
                dims = tf.reshape(
                    tf.stack([anchor_width, anchor_height], axis=-1), [1, 1, 2]
                )
                for scale in self.scales:
                    anchor_dims.append(scale * dims)
            anchor_dims_all.append(tf.stack(anchor_dims, axis=-2))
        return anchor_dims_all

    def _get_anchors(self, feature_height, feature_width, level):
        """Generates anchor boxes for a given feature map size and level

        Arguments:
          feature_height: An integer representing the height of the feature map.
          feature_width: An integer representing the width of the feature map.
          level: An integer representing the level of the feature map in the
            feature pyramid.

        Returns:
          anchor boxes with the shape
          `(feature_height * feature_width * num_anchors, 4)`
        """
        rx = tf.range(feature_width, dtype=tf.float32) + 0.5
        ry = tf.range(feature_height, dtype=tf.float32) + 0.5
        centers = tf.stack(tf.meshgrid(rx, ry), axis=-1) * \
            self._strides[level - 3]
        centers = tf.expand_dims(centers, axis=-2)
        centers = tf.tile(centers, [1, 1, self._num_anchors, 1])
        dims = tf.tile(
            self._anchor_dims[level - 3], [feature_height, feature_width, 1, 1]
        )
        anchors = tf.concat([centers, dims], axis=-1)
        return tf.reshape(
            anchors, [feature_height * feature_width * self._num_anchors, 4]
        )

    def get_anchors(self, image_height, image_width):
        """Generates anchor boxes for all the feature maps of the feature pyramid.

        Arguments:
          image_height: Height of the input image.
          image_width: Width of the input image.

        Returns:
          anchor boxes for all the feature maps, stacked as a single tensor
            with shape `(total_anchors, 4)`
        """
        anchors = [
            self._get_anchors(
                tf.math.ceil(image_height / 2 ** i),
                tf.math.ceil(image_width / 2 ** i),
                i,
            )
            for i in range(3, 8)
        ]
        return tf.concat(anchors, axis=0)


def convert_to_corners(boxes):
    """Changes the box format to corner coordinates

    Arguments:
      boxes: A tensor of rank 2 or higher with a shape of `(..., num_boxes, 4)`
        representing bounding boxes where each box is of the format
        `[x, y, width, height]`.

    Returns:
      converted boxes with shape same as that of boxes.
    """
    return tf.concat(
        [boxes[..., :2] - boxes[..., 2:] / 2.0,
            boxes[..., :2] + boxes[..., 2:] / 2.0],
        axis=-1,
    )


class DecodePredictions(tf.keras.layers.Layer):
    """A Keras layer that decodes predictions of the RetinaNet model.

    Attributes:
      num_classes: Number of classes in the dataset
      confidence_threshold: Minimum class probability, below which detections
        are pruned.
      nms_iou_threshold: IOU threshold for the NMS operation
      max_detections_per_class: Maximum number of detections to retain per
       class.
      max_detections: Maximum number of detections to retain across all
        classes.
      box_variance: The scaling factors used to scale the bounding box
        predictions.
    """

    def __init__(
        self,
        num_classes=80,
        confidence_threshold=0.05,
        nms_iou_threshold=0.5,
        max_detections_per_class=100,
        max_detections=100,
        box_variance=[0.1, 0.1, 0.2, 0.2],
        **kwargs
    ):
        super(DecodePredictions, self).__init__(**kwargs)
        self.num_classes = num_classes
        self.confidence_threshold = confidence_threshold
        self.nms_iou_threshold = nms_iou_threshold
        self.max_detections_per_class = max_detections_per_class
        self.max_detections = max_detections

        self._anchor_box = AnchorBox()
        self._box_variance = tf.convert_to_tensor(
            [0.1, 0.1, 0.2, 0.2], dtype=tf.float32
        )

    def _decode_box_predictions(self, anchor_boxes, box_predictions):
        boxes = box_predictions * self._box_variance
        boxes = tf.concat(
            [
                boxes[:, :, :2] * anchor_boxes[:, :, 2:] +
                anchor_boxes[:, :, :2],
                tf.math.exp(boxes[:, :, 2:]) * anchor_boxes[:, :, 2:],
            ],
            axis=-1,
        )
        boxes_transformed = convert_to_corners(boxes)
        return boxes_transformed

    def call(self, images, predictions):
        image_shape = tf.cast(tf.shape(images), dtype=tf.float32)
        anchor_boxes = self._anchor_box.get_anchors(
            image_shape[1], image_shape[2])
        box_predictions = predictions[:, :, :4]
        cls_predictions = tf.nn.sigmoid(predictions[:, :, 4:])
        boxes = self._decode_box_predictions(
            anchor_boxes[None, ...], box_predictions)

        return tf.image.combined_non_max_suppression(
            tf.expand_dims(boxes, axis=2),
            cls_predictions,
            self.max_detections_per_class,
            self.max_detections,
            self.nms_iou_threshold,
            self.confidence_threshold,
            clip_boxes=False,
        )

In [24]:
inp = tf.keras.Input(
    shape=(None, None, 3),
    name="input",
    dtype=tf.float32,
)

In [25]:
x = Resizing_Wilson()(inp)

In [26]:
x = Scale_Wilson()(x)

In [27]:
predictions = model(x)

In [28]:
detections = DecodePredictions(confidence_threshold=0.5)(inp, predictions)

In [29]:
modified_model = tf.keras.Model(inputs=inp, outputs=detections)

In [30]:
modified_model.summary()
# Calculae FLOPS
flops = get_flops(modified_model, batch_size=1)
print(f"FLOPS: {flops / 10 ** 9:.03} G")

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 resize_wilson (Resizing_Wilson  (None, 512, 512, 3)  0          ['input[0][0]']                  
 )                                                                                                
                                                                                                  
 scale_wilson (Scale_Wilson)    (None, 512, 512, 3)  0           ['resize_wilson[0][0]']          
                                                                                            

2022-10-25 18:23:53.753314: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-25 18:23:53.753642: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-25 18:23:53.753810: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 2
2022-10-25 18:23:53.753939: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2022-10-25 18:23:53.754390: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-25 18:23:53.754604: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] su


=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================

Doc:
scope: The nodes in the model graph are organized by their names, which is hierarchical like filesystem.
flops: Number of float operations. Note: Please read the implementation for the math behind it.

Profi

Incomplete shape.
Incomplete shape.


In [31]:
modified_model.save(
    filepath="/home/fsuser/AI_ENGINE/yolov5_tf_original/Yolov5/Ibrahim_Bhaai_model/latest_model",
    overwrite=True,
    include_optimizer=True,
    save_format='tf',
    save_traces=True
)

INFO:tensorflow:Assets written to: /home/fsuser/AI_ENGINE/yolov5_tf_original/Yolov5/Ibrahim_Bhaai_model/latest_model/assets


INFO:tensorflow:Assets written to: /home/fsuser/AI_ENGINE/yolov5_tf_original/Yolov5/Ibrahim_Bhaai_model/latest_model/assets


In [32]:
result = tf.math.top_k([1, 2, 98, 1, 1, 99, 3, 1, 3, 96, 4, 1],
                        k=3, sorted=True, name="talha")

In [33]:
result.count

<function TopKV2.count(value, /)>

In [34]:
result.count()

TypeError: tuple.count() takes exactly one argument (0 given)

In [35]:
dir(result)

['__add__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '_asdict',
 '_field_defaults',
 '_fields',
 '_make',
 '_replace',
 'count',
 'index',
 'indices',
 'values']

In [36]:
[k for k in dir(result) if not k.startswith("_")]

['count', 'index', 'indices', 'values']

In [37]:
result.count()

TypeError: tuple.count() takes exactly one argument (0 given)

In [38]:
result.indices

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([5, 2, 9], dtype=int32)>

In [39]:
result.values

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([99, 98, 96], dtype=int32)>

In [40]:
from_tensor(
    result.values, name=None
)

NameError: name 'from_tensor' is not defined

In [41]:
tf.TensorSpec.from_tensor(
    result.values, name=None
)

TensorSpec(shape=(3,), dtype=tf.int32, name=None)

In [42]:
tf.TensorSpec.from_tensor(
    result.values, name="talha"
)

TensorSpec(shape=(3,), dtype=tf.int32, name='talha')

In [43]:
batch_size = 10
num_boxes = 30
q = 1
num_classes = 80


boxes = tf.random.normal([[batch_size, num_boxes, q, 4]])
scores = tf.random.normal([batch_size, num_boxes, num_classes])

_boxes, _scores, _classes, _detections = tf.image.combined_non_max_suppression(
    boxes,
    scores,
    max_output_size_per_class=5,
    max_total_size=30,
    iou_threshold=0.5,
    score_threshold=0.05,
    pad_per_class=False,
    clip_boxes=True,
    name="talha_NMS"
)

2022-10-25 18:46:37.563737: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at random_op.cc:74 : INVALID_ARGUMENT: shape must be a vector of {int32,int64}, got shape [1,4]


InvalidArgumentError: shape must be a vector of {int32,int64}, got shape [1,4] [Op:RandomStandardNormal]

In [44]:
batch_size = 10
num_boxes = 30
q = 1
num_classes = 80


boxes = tf.random.normal([batch_size, num_boxes, q, 4])
scores = tf.random.normal([batch_size, num_boxes, num_classes])

_boxes, _scores, _classes, _detections = tf.image.combined_non_max_suppression(
    boxes,
    scores,
    max_output_size_per_class=5,
    max_total_size=30,
    iou_threshold=0.5,
    score_threshold=0.05,
    pad_per_class=False,
    clip_boxes=True,
    name="talha_NMS"
)

In [45]:
_boxes.shape

TensorShape([10, 30, 4])

In [46]:
_detections

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([30, 30, 30, 30, 30, 30, 30, 30, 30, 30], dtype=int32)>

In [47]:
_scores

<tf.Tensor: shape=(10, 30), dtype=float32, numpy=
array([[2.8564093, 2.84841  , 2.8361828, 2.8156302, 2.8118753, 2.7423787,
        2.696819 , 2.6818495, 2.5866737, 2.5225058, 2.5192664, 2.5179677,
        2.4885209, 2.481537 , 2.459589 , 2.4428022, 2.4388711, 2.4340494,
        2.4317486, 2.383421 , 2.324988 , 2.3189652, 2.317647 , 2.2745988,
        2.274024 , 2.2701085, 2.2537985, 2.2389438, 2.2347877, 2.2212107],
       [3.5670855, 3.4510534, 3.188483 , 3.0203934, 2.937743 , 2.6980464,
        2.6808503, 2.5364466, 2.5112221, 2.4818962, 2.4791028, 2.4774296,
        2.476006 , 2.4511976, 2.4511886, 2.4111307, 2.3977425, 2.3972373,
        2.3942673, 2.3899906, 2.3734794, 2.3468306, 2.3232276, 2.3200455,
        2.3149257, 2.2815287, 2.2755125, 2.2589812, 2.2399702, 2.2155938],
       [3.8350198, 3.3156426, 3.0942464, 2.983915 , 2.8887062, 2.8836253,
        2.836785 , 2.8108854, 2.7898815, 2.7237422, 2.6760433, 2.667378 ,
        2.5967944, 2.544631 , 2.5393262, 2.5342286, 2.507340

In [48]:
_scores[0]

<tf.Tensor: shape=(30,), dtype=float32, numpy=
array([2.8564093, 2.84841  , 2.8361828, 2.8156302, 2.8118753, 2.7423787,
       2.696819 , 2.6818495, 2.5866737, 2.5225058, 2.5192664, 2.5179677,
       2.4885209, 2.481537 , 2.459589 , 2.4428022, 2.4388711, 2.4340494,
       2.4317486, 2.383421 , 2.324988 , 2.3189652, 2.317647 , 2.2745988,
       2.274024 , 2.2701085, 2.2537985, 2.2389438, 2.2347877, 2.2212107],
      dtype=float32)>

In [49]:
boxes = tf.constant([[[10,20,30,40]],[[10,20,30,40]],[[10,20,30,40]]])

In [50]:
boxes.shape

TensorShape([3, 1, 4])

In [51]:
boxes = tf.constant([[[[10,20,30,40]],[[10,20,30,40]],[[10,20,30,40]]]])

In [52]:
boxes.shape

TensorShape([1, 3, 1, 4])

In [53]:
scores = tf.constant(
    [
        [
            [0.3, 0.2, 0.8],
            [0.5, 0.6,0.7],
            [0.9,0.1,0.7]
        ]
    ]
)

In [54]:
scores.shape

TensorShape([1, 3, 3])

In [55]:
_boxes, _scores, _classes, _detections = tf.image.combined_non_max_suppression(
    boxes,
    scores,
    max_output_size_per_class=5,
    max_total_size=30,
    iou_threshold=0.5,
    score_threshold=0.05,
    pad_per_class=False,
    clip_boxes=True,
    name="talha_NMS"
)

InvalidArgumentError: cannot compute CombinedNonMaxSuppression as input #0(zero-based) was expected to be a float tensor but is a int32 tensor [Op:CombinedNonMaxSuppression]

In [56]:
_boxes, _scores, _classes, _detections = tf.image.combined_non_max_suppression(
    tf.cast(boxes, tf.float32),
    tf.cast(scores, tf.float32),
    max_output_size_per_class=5,
    max_total_size=30,
    iou_threshold=0.5,
    score_threshold=0.05,
    pad_per_class=False,
    clip_boxes=True,
    name="talha_NMS"
)

In [57]:
_boxes

<tf.Tensor: shape=(1, 30, 4), dtype=float32, numpy=
array([[[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]], dtype=float32)>

In [58]:
_boxes, _scores, _classes, _detections = tf.image.combined_non_max_suppression(
    tf.cast(boxes, tf.float32),
    tf.cast(scores, tf.float32),
    max_output_size_per_class=2,
    max_total_size=5,
    iou_threshold=0.5,
    score_threshold=0.05,
    pad_per_class=False,
    clip_boxes=True,
    name="talha_NMS"
)

In [59]:
_detections

<tf.Tensor: shape=(1,), dtype=int32, numpy=array([3], dtype=int32)>

In [60]:
_detections

<tf.Tensor: shape=(1,), dtype=int32, numpy=array([3], dtype=int32)>

In [61]:
_scores

<tf.Tensor: shape=(1, 5), dtype=float32, numpy=array([[0.9, 0.8, 0.6, 0. , 0. ]], dtype=float32)>

In [62]:
_scores

<tf.Tensor: shape=(1, 5), dtype=float32, numpy=array([[0.9, 0.8, 0.6, 0. , 0. ]], dtype=float32)>

In [63]:
tf.math.top_k(
    _scores, k=1, sorted=True, name="decreasing_scores"
)

TopKV2(values=<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.9]], dtype=float32)>, indices=<tf.Tensor: shape=(1, 1), dtype=int32, numpy=array([[0]], dtype=int32)>)

In [64]:
results = tf.math.top_k(
    _scores, k=2, sorted=True, name="decreasing_scores"
)

In [65]:
results.

SyntaxError: invalid syntax (<ipython-input-65-1261788e8061>, line 1)

In [66]:
results.values

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0.9, 0.8]], dtype=float32)>

In [67]:
results.indices

<tf.Tensor: shape=(1, 2), dtype=int32, numpy=array([[0, 1]], dtype=int32)>

In [68]:
class_names = tf.constant(["class1", "class2", "class3"])

In [69]:
class_names[0]

<tf.Tensor: shape=(), dtype=string, numpy=b'class1'>

In [70]:
tf.gather(class_names, results.indices)

<tf.Tensor: shape=(1, 2), dtype=string, numpy=array([[b'class1', b'class2']], dtype=object)>

In [71]:
image_random = tf.random.normal([1, 800, 800, 3])

In [72]:
out = modified_model.predict(image_random)

2022-10-25 19:41:52.515809: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8300
2022-10-25 19:41:54.116979: W tensorflow/core/framework/op_kernel.cc:1733] INVALID_ARGUMENT: required broadcastable shapes
2022-10-25 19:41:54.117285: W tensorflow/core/framework/op_kernel.cc:1733] INVALID_ARGUMENT: required broadcastable shapes


InvalidArgumentError: Graph execution error:

Detected at node 'model_1/decode_predictions_1/mul_16' defined at (most recent call last):
    File "/home/fsuser/miniconda3/envs/ai_engine/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/fsuser/miniconda3/envs/ai_engine/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/fsuser/miniconda3/envs/ai_engine/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/fsuser/miniconda3/envs/ai_engine/lib/python3.9/site-packages/traitlets/config/application.py", line 976, in launch_instance
      app.start()
    File "/home/fsuser/miniconda3/envs/ai_engine/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/home/fsuser/miniconda3/envs/ai_engine/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 199, in s

In [73]:
image_random = tf.random.normal([1, 500, 500, 3])

In [74]:
out = modified_model.predict(image_random)

2022-10-25 19:42:42.711436: W tensorflow/core/framework/op_kernel.cc:1733] INVALID_ARGUMENT: required broadcastable shapes
2022-10-25 19:42:42.711548: W tensorflow/core/framework/op_kernel.cc:1733] INVALID_ARGUMENT: required broadcastable shapes


InvalidArgumentError: Graph execution error:

Detected at node 'model_1/decode_predictions_1/mul_16' defined at (most recent call last):
    File "/home/fsuser/miniconda3/envs/ai_engine/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/fsuser/miniconda3/envs/ai_engine/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/fsuser/miniconda3/envs/ai_engine/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/fsuser/miniconda3/envs/ai_engine/lib/python3.9/site-packages/traitlets/config/application.py", line 976, in launch_instance
      app.start()
    File "/home/fsuser/miniconda3/envs/ai_engine/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/home/fsuser/miniconda3/envs/ai_engine/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 199, in s

In [75]:
out = modified_model(image_random)

2022-10-25 19:42:51.959410: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2022-10-25 19:42:52.384766: W tensorflow/core/framework/op_kernel.cc:1733] INVALID_ARGUMENT: required broadcastable shapes


InvalidArgumentError: Exception encountered when calling layer "decode_predictions_1" (type DecodePredictions).

required broadcastable shapes [Op:Mul]

Call arguments received:
  • images=tf.Tensor(shape=(1, 500, 500, 3), dtype=float32)
  • predictions=tf.Tensor(shape=(1, 49104, 24), dtype=float32)

In [76]:
modified_model.get_config

<bound method Functional.get_config of <keras.engine.functional.Functional object at 0x7f0bf530c490>>

In [77]:
modified_model.get_config()

NotImplementedError: 

In [78]:
model.get_config()

NotImplementedError: 

In [79]:
detection_model.get_config()

NotImplementedError: 

In [80]:
out = modified_model.inputs

In [81]:
out

[<KerasTensor: shape=(None, None, None, 3) dtype=float32 (created by layer 'input')>]

In [82]:
out = modified_model.predict(image_random)

2022-10-25 19:44:33.723255: W tensorflow/core/framework/op_kernel.cc:1733] INVALID_ARGUMENT: required broadcastable shapes
2022-10-25 19:44:33.723372: W tensorflow/core/framework/op_kernel.cc:1733] INVALID_ARGUMENT: required broadcastable shapes


InvalidArgumentError: Graph execution error:

Detected at node 'model_1/decode_predictions_1/mul_16' defined at (most recent call last):
    File "/home/fsuser/miniconda3/envs/ai_engine/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/fsuser/miniconda3/envs/ai_engine/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/fsuser/miniconda3/envs/ai_engine/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/fsuser/miniconda3/envs/ai_engine/lib/python3.9/site-packages/traitlets/config/application.py", line 976, in launch_instance
      app.start()
    File "/home/fsuser/miniconda3/envs/ai_engine/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/home/fsuser/miniconda3/envs/ai_engine/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 199, in s

In [83]:
image_random = tf.random.normal([1, 512, 512, 3])

In [84]:
out = modified_model.predict(image_random)

In [85]:
out

CombinedNonMaxSuppression(nmsed_boxes=array([[[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],


In [86]:
len(out)

4

In [87]:
boxes, scores, classes, detections = out

In [88]:
boxes.shape

(1, 100, 4)

In [89]:
fitlered_scores = tf.math.top_k(
    scores, k=10, sorted=True, name="selected_scores"
)

In [90]:
fitlered_scores

TopKV2(values=<tf.Tensor: shape=(1, 10), dtype=float32, numpy=array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>, indices=<tf.Tensor: shape=(1, 10), dtype=int32, numpy=array([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]], dtype=int32)>)

In [91]:
filtered_boxes = tf.gather(boxes, fitlered_scores.indices)

In [92]:
filtered_boxes

<tf.Tensor: shape=(1, 10, 100, 4), dtype=float32, numpy=
array([[[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         ...,
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         ...,
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         ...,
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        ...,

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         ...,
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         ...,
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0.

In [93]:
filtered_boxes.shape

TensorShape([1, 10, 100, 4])

In [94]:
classes

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]], dtype=float32)

In [95]:
classes.shape

(1, 100)

In [96]:
fitlered_scores

TopKV2(values=<tf.Tensor: shape=(1, 10), dtype=float32, numpy=array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>, indices=<tf.Tensor: shape=(1, 10), dtype=int32, numpy=array([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]], dtype=int32)>)

In [97]:
fitlered_scores.indices.shape

TensorShape([1, 10])

In [98]:
tf.gather(classes, fitlered_scores.indices)

<tf.Tensor: shape=(1, 10, 100), dtype=float32, numpy=
array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.],
        [0., 0., 0

In [99]:
tf.gather(classes, fitlered_scores.indices).shape

TensorShape([1, 10, 100])